In [ ]:
# Aggregate statistics of all results
import os
import json


llm_output_dir = './llm_outputs/'
output_num = 0
file_num = 0
prompt_types = ['0_shot_cot_restrict', '0_shot_cot', '0_shot_restrict', '0_shot', '5_shot_cot_restrict', '5_shot_cot', '5_shot_restrict', '5_shot']
prompt_type_file_count = {i: 0 for i in prompt_types}
prompt_type_count = {i: 0 for i in prompt_types}
models = ['GPT_4o', 'LLaMA3.1-8B', 'DeepSeek-R1-Distill-Llama-8B', 'DeepSeek-R1-Distill-Qwen-1.5B', 'DeepSeek-R1-Distill-Qwen-7B', 'DeepSeek-R1-Distill-Qwen-14B', 'Qwen2-7B-Instruct', 'Qwen2-1.5B-Instruct', 'Qwen2.5-7B-Instruct', 'Qwen2.5-14B-Instruct', 'InternLM2.5-7B-Chat', 'ChatGLM3-6B', 'GLM-4-9B-Chat', 'Gemma-2-9B-it', 'Gemma-2-2B-it', "Llama-3.2-1B-Instruct", "Llama-3.2-3B-Instruct", "QwQ-32B-2", "Phi-4"]
model_file_count = {i: 0 for i in models}
model_count = {i: 0 for i in models}
model_output_avg_length = {i: [0, 0] for i in models}
datasets = ['MMLU-Redux_enh', 'MMLU-Redux', 'MMLU-Pro_enh', 'MMLU-Pro', 'DROP', 'GPQA_enh', 'GPQA', 'C-SimpleQA', 'FRAMES', 'AIME_2024', 'LiveMathBench_zh', 'LiveMathBench_en', 'CLUEWSC', 'C-Eval_enh', 'C-Eval', 'SimpleQA', 'AMC23', 'OlympiadBench_en', 'OlympiadBench_zh', 'MGSM', 'CMATH', 'GSM8K', 'MATH', 'AgNews', 'Amazon', 'CMNLI', 'ARC', 'CHID', 'CMMLU_enh', 'CMMLU', 'MMLU_enh', 'MMLU']
dataset_file_count = {i: 0 for i in datasets}
dataset_count = {i: 0 for i in datasets}
key_answer_type_count = {
    "alphabet_option": 0,
    "math": 0,
    "short_text": 0,
    "categorical_label": 0
}
output_len_ls = []
for file in os.listdir(llm_output_dir):
    if not file.endswith('json'):
        continue
    file_path = os.path.join(llm_output_dir, file)
    with open(file_path, 'r') as f:
        data = json.load(f)
    data_num = len(data['results'])
    data_output_len_ls = []
    for item in data['results']:
        data_output_len_ls.append(len(item['llm_output']))
        key_answer_type_count[item["key_answer_type"]] += 1
    output_num += len(data['results'])
    for prompt_type in prompt_type_file_count:
        if file.startswith(prompt_type):
            prompt_type_file_count[prompt_type] += 1
            prompt_type_count[prompt_type] += data_num
            break
    for model in model_file_count:
        if file.__contains__(model):
            model_file_count[model] += 1
            model_count[model] += data_num
            model_output_avg_length[model][0] += len(data['results'])
            model_output_avg_length[model][1] += sum(data_output_len_ls)
            break
    for dataset in dataset_file_count:
        if file.__contains__(dataset):
            dataset_file_count[dataset] += 1
            dataset_count[dataset] += data_num
            break
    file_num += 1
    output_len_ls.extend(data_output_len_ls)

for model in models:
    model_output_avg_length[model] = model_output_avg_length[model][1] / model_output_avg_length[model][0]

In [ ]:
file_num, output_num

In [ ]:
key_answer_type_count

In [ ]:
model_count, model_file_count

In [ ]:
dataset_count, dataset_file_count

In [ ]:
prompt_type_count, prompt_type_file_count

In [ ]:
model_output_avg_length

In [ ]:
print(
    " length < 1000: ", len([i for i in output_len_ls if i < 1000]), '\n',
    "1000 <= length < 2000: ", len([i for i in output_len_ls if 1000 <= i < 2000]), '\n',
    "2000 <= length < 3000: ", len([i for i in output_len_ls if 2000 <= i < 3000]), '\n',
    "3000 <= length < 4000: ", len([i for i in output_len_ls if 3000 <= i < 4000]), '\n',
    "4000 <= length < 5000: ", len([i for i in output_len_ls if 4000 <= i < 5000]), '\n',
    "5000 <= length < 6000: ", len([i for i in output_len_ls if 5000 <= i < 6000]), '\n',
    "6000 <= length: ", len([i for i in output_len_ls if 6000 <= i]), '\n',
    "avg length:", round(sum(output_len_ls) / len(output_len_ls), 2)
)

In [ ]:
# Consolidate the outcomes from all large models' responses
import os
import json


llm_output_dir = './llm_outputs/'
output_all = []
for file in os.listdir(llm_output_dir):
    if not file.endswith('json'):
        continue
    file_path = os.path.join(llm_output_dir, file)
    with open(file_path, 'r') as f:
        data = json.load(f)
    setting = data['info']['setting']
    model_name = data['info']['llm']['model_name']
    for item in data['results']:
        item_ = item.copy()
        del item_['prompt']
        del item_['index']
        output_all.append(
            {
                "setting": setting,
                "model_name": model_name,
                **item_
            }
        )

with open('./processed_outputs/output_all.json', 'w', encoding='utf-8') as f:
    json.dump(output_all, f, ensure_ascii=False, indent=4)

In [ ]:
# Filter samples
import os
import json
import random


random.seed(42)


data_path = './processed_outputs/output_all.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

alphabet_data = []
math_data = []
short_text_data = []
categorical_data = []

for item in data:
    if item["key_answer_type"] == "alphabet_option":
        alphabet_data.append(item)
    elif item["key_answer_type"] == "math":
        math_data.append(item)
    elif item["key_answer_type"] == "short_text":
        short_text_data.append(item)
    elif item["key_answer_type"] == "categorical_label":
        categorical_data.append(item)

random.shuffle(alphabet_data)
random.shuffle(math_data)
random.shuffle(short_text_data)
# random.shuffle(categorical_data)

# print(len(alphabet_data), len(math_data), len(short_text_data), len(categorical_data))

output_data_filtered = alphabet_data[:8000] + \
                       math_data[:8000] + \
                       short_text_data[:8000] + \
                       categorical_data[:8000]

output_data_remaining1 = alphabet_data[8000:] + \
                         math_data[8000:] + \
                         short_text_data[8000:] + \
                         categorical_data[:8000]


output_data_remaining2 = []
for item in output_data_remaining1:
    if len(item['llm_output']) >= 3000:
        flag = random.choice([0, 1])
        if flag == 1:
            output_data_filtered.append(item)
        else:
            output_data_remaining2.append(item)
    else:
        output_data_remaining2.append(item)

output_data_filtered_path = './processed_outputs/output_data_filtered.json'
with open(output_data_filtered_path, 'w', encoding='utf-8') as f:
    json.dump(output_data_filtered, f, ensure_ascii=False, indent=4)
output_data_remaining_path = './processed_outputs/output_data_remaining.json'
with open(output_data_remaining_path, 'w', encoding='utf-8') as f:
    json.dump(output_data_remaining2, f, ensure_ascii=False, indent=4)

In [ ]:
len(output_data_filtered), len(output_data_remaining2)

In [ ]:
# Statistical information of the filtered dataset
output_num = 0

prompt_types = ['0_shot_cot_restrict', '0_shot_cot', '0_shot_restrict', '0_shot', '5_shot_cot_restrict', '5_shot_cot', '5_shot_restrict', '5_shot']
prompt_type_count = {i: 0 for i in prompt_types}

models = ['GPT_4o', 'LLaMA3.1-8B', 'DeepSeek-R1-Distill-Llama-8B', 'DeepSeek-R1-Distill-Qwen-1.5B', 'DeepSeek-R1-Distill-Qwen-7B', 'DeepSeek-R1-Distill-Qwen-14B', 'Qwen2-7B-Instruct', 'Qwen2-1.5B-Instruct', 'Qwen2.5-7B-Instruct', 'Qwen2.5-14B-Instruct', 'InternLM2.5-7B-Chat', 'ChatGLM3-6B', 'GLM-4-9B-Chat', 'Gemma-2-9B-it', 'Gemma-2-2B-it', "Llama-3.2-1B-Instruct", "Llama-3.2-3B-Instruct", "QwQ-32B-2", "Phi-4"]
model_count = {i: 0 for i in models}

model_output_avg_length = {i: 0 for i in models}

datasets = ['MMLU-Redux_enh', 'MMLU-Redux', 'MMLU-Pro_enh', 'MMLU-Pro', 'DROP', 'GPQA_enh', 'GPQA', 'C-SimpleQA', 'FRAMES', 'AIME_2024', 'LiveMathBench_zh', 'LiveMathBench_en', 'CLUEWSC', 'C-Eval_enh', 'C-Eval', 'SimpleQA', 'AMC23', 'OlympiadBench_en', 'OlympiadBench_zh', 'MGSM', 'CMATH', 'GSM8K', 'MATH', 'AgNews', 'Amazon', 'CMNLI', 'ARC', 'CHID', 'CMMLU_enh', 'CMMLU', 'MMLU_enh', 'MMLU']
dataset_count = {i: 0 for i in datasets}

key_answer_type_count = {
    "alphabet_option": 0,
    "math": 0,
    "short_text": 0,
    "categorical_label": 0
}
output_len_ls = []

for item in output_data_filtered:
    prompt_type_count[item['setting']] += 1
    model_count[item['model_name']] += 1
    output_length = len(item['llm_output'])
    model_output_avg_length[item['model_name']] += output_length
    output_len_ls.append(output_length)
    dataset_count[item['dataset']] += 1
    key_answer_type_count[item['key_answer_type']] += 1

for model in models:
    model_output_avg_length[model] = model_output_avg_length[model] / model_count[model]


In [ ]:
prompt_type_count

In [ ]:
model_count

In [ ]:
model_output_avg_length

In [ ]:
key_answer_type_count

In [ ]:
dataset_count

In [ ]:
print(
    " length < 1000: ", len([i for i in output_len_ls if i < 1000]), '\n',
    "1000 <= length < 2000: ", len([i for i in output_len_ls if 1000 <= i < 2000]), '\n',
    "2000 <= length < 3000: ", len([i for i in output_len_ls if 2000 <= i < 3000]), '\n',
    "3000 <= length < 4000: ", len([i for i in output_len_ls if 3000 <= i < 4000]), '\n',
    "4000 <= length < 5000: ", len([i for i in output_len_ls if 4000 <= i < 5000]), '\n',
    "5000 <= length < 6000: ", len([i for i in output_len_ls if 5000 <= i < 6000]), '\n',
    "6000 <= length: ", len([i for i in output_len_ls if 6000 <= i]), '\n',
    "avg length:", round(sum(output_len_ls) / len(output_len_ls), 2)
)

In [ ]:
# Compare the consistency of two GPT-4o annotations.
import json
import re


def extract_judgement(text):
    if type(text) != str:
        print('None: ', text)
        return "None"
    match = re.search(r'"judgement": "(Incorrect|Correct)"', text)
    if match:
        return match.group(1)
    else:
        print('Not match: ', text)
        return "None"


def extract(s):
    try:
        s = s[s.find('{'):s.rfind('}') + 1]
        s = json.loads(s)['judgement']
    except Exception as e:
        s = s.split('"judgement": ')[-1]
        s = s[s.find('"') + 1:s.rfind('"')]
    
    return s

# Two GPT-4o annotation result files
judge_result_path1 = ""
judge_result_path2 = ""
data_path = "./processed_outputs/output_data_filtered.json"

with open(judge_result_path1, 'r') as f:
    judge_result1 = json.load(f)

with open(judge_result_path2, 'r') as f:
    judge_result2 = json.load(f)


with open(data_path, 'r') as f:
    data = json.load(f)

print(len(judge_result1['results']), 
      len(judge_result2['results']), 
      len(data))
num = len(judge_result1['results'])
not_same_num = 0
for i in range(num):
    item1 = judge_result1['results'][i]
    item2 = judge_result2['results'][i]
    assert item1['llm_output'] == item2['llm_output']
    assert item1['llm_output'] == data[i]['llm_output']
    judge1 = extract_judgement(item1['GPT_4o_judgment_result'])
    judge2 = extract_judgement(item2['GPT_4o_judgment_result'])
    data[i]['GPT_4o_judgment_result1'] = judge1
    data[i]['GPT_4o_judgment_result2'] = judge2
    data[i]['GPT_4o_judgment_consistency'] = "True"
    data[i]['GPT_4o_final_judgment_result'] = judge1
    if judge1 != judge2:
        not_same_num += 1
        data[i]['GPT_4o_judgment_consistency'] = "False"
        data[i]['GPT_4o_final_judgment_result'] = ""

# print("Inconsistent sample number: ", not_same_num)

output_path = './processed_outputs/output_data_filtered_judge.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
# Extract samples where GPT-4o's judgments are inconsistent across two runs, as well as all math problem samples, for subsequent manual annotation.
import json


data_path = './processed_outputs/output_data_filtered_judge.json'
with open(data_path, 'r') as f:
    data = json.load(f)

human_label_data = []

for item in data:
    if item['GPT_4o_judgment_consistency'] == "False" or item['key_answer_type'] == "math":
        human_label_data.append(item)

print(len(human_label_data))
output_path = './processed_outputs/output_data_filtered_judge_need_human_label.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(human_label_data, f, ensure_ascii=False, indent=4)